# Predicciones de fraudes bancarios
Hoy vamos a predecir si transacciones de un data frame obtenido en [Kaggle](https://www.kaggle.com/datasets/dhanushnarayananr/credit-card-fraud/data) son legítimas o no. Vamos a utilizar dos modelos y ponerlos frente a frente, para así explicar sus diferencias y cuál es más efectivo y acertado en nuestra tarea. Pondremos a prueba `LogisticRegression` con `RandomForestClassifier`. ¿Cuál crees que sea más robusto para nuestra tarea?

## Familiarización con los datos
Vamos a familiarizarnos con el _dataset_.

In [5]:
import pandas as pd

df = pd.read_csv('card_transdata.csv')
display(df.head(3))

display(df.info())

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB


None

Tenemos ocho columnas y un millón de entradas. Cinco columnas son de datos binarios y tres de números decimales. Tenemos como características la distancia de la orden desde la casa del cliente, si uso un número PIN, si fue una venta en línea, la distancia desde la última transacción, etc. No hay datos nulos.

Es un conjunto de datos listo para trabajar.

## Separación de características y variable objetivo, y entrenamiento y prueba
Como sabes, debemos trabajar con las características y la variable objetivo.

In [6]:
X = df.drop('fraud', axis=1)  # Todas las columnas (características), menos la variable objetivo.
y = df['fraud']               # La variable objetivo, la columna 'fraud'

Ahora vamos a dividir entre datos de entrenamiento y datos de prueba:

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y        
)

Con la información dividida, estamos listos para probar nuestro primer modelo, el cual será `LogisticRegression`.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

log_reg = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    n_jobs=-1
)

log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)

print("Logistic Regression - matriz de confusión:")
print(confusion_matrix(y_test, y_pred_lr))
print("\nLogistic Regression - reporte de clasificación:")
print(classification_report(y_test, y_pred_lr, digits=4))


Logistic Regression - matriz de confusión:
[[255484  18295]
 [  1356  24865]]

Logistic Regression - reporte de clasificación:
              precision    recall  f1-score   support

         0.0     0.9947    0.9332    0.9630    273779
         1.0     0.5761    0.9483    0.7168     26221

    accuracy                         0.9345    300000
   macro avg     0.7854    0.9407    0.8399    300000
weighted avg     0.9581    0.9345    0.9414    300000



El modelo de regresión logística funciona especialmente bien para la clase de fraude (1), lo cuál es el objetivo principal en un modelo como éste. 

Ahora es turno de `RandomForestClassifier`.

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced_subsample'  # ayuda con el desbalance
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("Random Forest - matriz de confusión:")
print(confusion_matrix(y_test, y_pred_rf))
print("\nRandom Forest - reporte de clasificación:")
print(classification_report(y_test, y_pred_rf, digits=4))


Random Forest - matriz de confusión:
[[273779      0]
 [     5  26216]]

Random Forest - reporte de clasificación:
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000    273779
         1.0     1.0000    0.9998    0.9999     26221

    accuracy                         1.0000    300000
   macro avg     1.0000    0.9999    0.9999    300000
weighted avg     1.0000    1.0000    1.0000    300000



El modelo _Random Forest_ funciona casi de manera perfecta. Detecta prácticamente todos los fraudes y casi nunca se equivoca marcando una transacción normal como fraude.

Finalmente, vamos a comparar _ROC-AUC_ de ambos.

In [10]:
from sklearn.metrics import roc_auc_score

y_proba_lr = log_reg.predict_proba(X_test)[:, 1]
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("ROC-AUC Logistic Regression:", roc_auc_score(y_test, y_proba_lr))
print("ROC-AUC Random Forest:", roc_auc_score(y_test, y_proba_rf))


ROC-AUC Logistic Regression: 0.9795724082321173
ROC-AUC Random Forest: 0.9999999974926029


El _Random Forest_ supera claramente a la _Logistic Regression_ y se comporta casi como un clasificador perfecto; esto refuerza lo que ya sugerían la matriz de confusión y el reporte de clasificación.

## Conclusión
El modelo de _Regresión Logística_ ofrece un desempeño sólido, pero el _Random Forest_ resulta claramente superior: logra detectar prácticamente todos los fraudes y casi no se equivoca con transacciones legítimas, con métricas cercanas a la perfección.